In [ ]:

import numpy as np

R0, Z0 = 1.0, 0.0
BPhi0 = 2.5

m = 1 # toroidal turn number

def cycleRZ(phi):
    return np.array([
        R0, Z0
    ])
# def cycleRZ(phi):
#     Rax, Zax = 1.0, 0.0
#     iota, phi0 = 1/3, 0.0
#     Rell, Zell = 0.5, 1.0
#     return np.array([
#         Rell * np.cos(iota*phi+phi0) + Rax,
#         Zell * np.sin(iota*phi+phi0) + Zax
#     ])

dTETdphi = 0.5
def TETu(phi):
    return np.pi/2 + phi/2
def TETs(phi):
    return phi/2

lamu = -np.e**(1/3)
lams = -np.e**(-1/3)

def BRBZ(phi, X_pol):
    V = np.array([
        [np.cos(TETu(phi) ), np.cos(TETs(phi) )],
        [np.sin(TETu(phi) ), np.sin(TETs(phi) )],
    ])
    Lam = np.array([
        [np.log( np.abs(lamu) )/(2*m*np.pi), 0],
        [0, np.log( np.abs(lams) )/(2*m*np.pi)],
    ])
    
    A = V @ Lam @ np.linalg.inv(V) + np.array([
        [0.0, -dTETdphi],
        [dTETdphi, 0.0],
    ])
    
    return BPhi0/R0 * A @ (X_pol-cycleRZ(phi))

def BPhi(phi, X_pol):
    return R0 * BPhi0 / X_pol[0]
def frac_RBpol_BPhi(phi, X_pol):
    return X_pol[0] * BRBZ(phi, X_pol) / BPhi(phi, X_pol)

In [ ]:
from scipy.integrate import solve_ivp
cylflt = solve_ivp(frac_RBpol_BPhi, [0, 2*3*np.pi], [1.0, 0.0+5e-2], dense_output=True, max_step=2*np.pi/10000)

In [ ]:
from plotly.graph_objs import Scatter3d
from plotly.subplots import make_subplots
import numpy as np

import plotly.graph_objects as go

nPhi, nS = 200, 40
Phi = np.linspace(0, 2*np.pi, num=nPhi, endpoint=True)
TETu = np.pi/2 + Phi / 2 
TETs = Phi / 2 
R0, Z0 = 1.0, 0.0

fig = make_subplots(
    rows=1, cols=1,
    specs=[[{'is_3d': True}, ]],
)






R_isphi = R0 + np.linspace(0.0, 0.3, num=nS)[:,None] * np.cos(TETu)[None,:] # [iS, iPhi]
Z_isphi = Z0 + np.linspace(0.0, 0.3, num=nS)[:,None] * np.sin(TETu)[None,:] # [iS, iPhi]
S_isphi = np.linspace(0.0, 0.3, num=nS)[:,None] * np.ones_like(TETu)[None,:] # [iS, iPhi]
x_isphi = R_isphi*np.cos(Phi)[None,:]
y_isphi = R_isphi*np.sin(Phi)[None,:] 
z_isphi = Z_isphi

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=S_isphi, 
        colorscale="Reds_r",
        opacityscale=[
            [0.0, 0.8], # vertex with min value is totally opaque  
#             [0.1, 0.9],
#             [0.2, 0.7],
            [0.5, 0.8],
            [1.0, 0.0] # vertex with max value is totally transparent  
        ],
    ),1,1, )


R_isphi = R0 + np.linspace(0.0, 0.3, num=nS)[:,None] * np.cos(TETs)[None,:] # [iS, iPhi]
Z_isphi = Z0 + np.linspace(0.0, 0.3, num=nS)[:,None] * np.sin(TETs)[None,:] # [iS, iPhi]
S_isphi = np.linspace(0.0, 0.3, num=nS)[:,None] * np.ones_like(TETs)[None,:] # [iS, iPhi]
x_isphi = R_isphi*np.cos(Phi)[None,:]
y_isphi = R_isphi*np.sin(Phi)[None,:] 
z_isphi = Z_isphi

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=S_isphi, 
        colorscale="Blues_r",
        opacityscale=[
            [0.0, 0.8], # vertex with min value is totally opaque  
#             [0.1, 0.9],
#             [0.2, 0.7],
            [0.5, 0.8],
            [1.0, 0.0] # vertex with max value is totally transparent  
        ],
    ),1,1, )



fig.add_trace(    
    go.Scatter3d(
        x=cylflt.sol(Phi*3)[0,:]*np.cos(Phi*3), 
        y=cylflt.sol(Phi*3)[0,:]*np.sin(Phi*3), 
        z=cylflt.sol(Phi*3)[1,:], mode="lines", line=dict(color='black', width=3) ),               
)



fig.show()